# Task 1
## Parsing the pdf resume into a database with the following fields:

•	First Name 

•	Last Name

•	Education

•	Date of Degree(s)


In [1]:
## Python packages

import glob  
import os
import re
import sys
import pandas as pd

from tika import parser
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
stop_words = stopwords.words('english')

## Python packages for Database- you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

# Resume Info Extractor

In [2]:
def user_df(resume, column_headings, personalId):
    
    """Create a Pandas DataFrame for the user infromation in the resume.
        input
    ------------
    resume: all of the text Tika parses from the resume 
    column_headings: the list of column headings for the DataFrame
    """

    #Tokenizing and filtering the resume off stopwords and punctuations 
    tokens = word_tokenize(resume)
    #storing the cleaned resume
    filtered = [w for w in tokens if w not in stop_words and w not in string.punctuation]
    #get the name from the resume
    first_name  = str(filtered[2])
    last_name = str(filtered[5])
    print "Name : " + first_name + ' ' +last_name

    # Using regular expressions to extract email address
    email = ""
    match_email = re.search(r'[\w\.-]+@[\w\.-]+', resume) 
    email = match_email.group(0)
    print email
    
    personal_info = [[personalId, first_name, last_name, email]]
    
    df = pd.DataFrame(personal_info, columns=column_headings)
    return df 

In [3]:
def education_df(resume, pattern_wi_content, pattern_wi_line, column_headings, personalId):
    
    """ Create a Pandas DataFrame for the Education section in the resume.

        input
    ------------
        
    resume: all of the text Tika parses from the resume 
    content_pattern: a pattern that identifies the set of lines that will become rows in the DataFrame
    line_pattern: a pattern within the content_pattern
    column_headings: the list of column headings for the DataFrame """

    # Grab all of the lines of text that match the pattern in content_pattern
    matched = re.search(pattern_wi_content, resume, re.DOTALL)
    # group(1): only keep the lines between the parentheses in the pattern
    matched = matched.group(1)
    # Split on newlines to create a sequence of strings
    matched = matched.split('\n')
    # Iterate over each line
    
    lines_list = []
    for item in matched:
        
        if bool(re.search(r'\d', item)):
                
                line_match = re.search(pattern_wi_line, item)

                from_year = line_match.group(1)
                to_year = line_match.group(3)
                school = line_match.group(4)
                state = line_match.group(5)
                
                lines_list.append([personalId, school, state, from_year, to_year])
                
                       
    # Convert the list of lists into a Pandas DataFrame and specify the column headings
    df = pd.DataFrame(lines_list, columns=column_headings)
    return df 

In [4]:
# In the Education table in the resume, grab all of the lines between Education and RESEARCH INTERESTS
education_pattern = r'EDUCATION(.*?)\nRESEARCH INTERESTS'

# For the education, grab the college name, years and state
college_pattern = r'([\d]{4}).*?([\S]{6})?\s+([\d]{4})?\s+([\S\s]+)\s+(\w+,\s\w+)'


# Column headings for the Education DataFrames
college_columns = ['ID', 'School', 'State', 'From_year', 'To_year']
user_columns = ['ID', 'First Name', 'Last Name', 'Email']

# Iterate over all PDF files in the folder and process each one in turn
for input_file in glob.glob(os.path.join('/Users/mohammad/Desktop/Vanguard/Resume_Folder', '*.pdf')):
    # Grab the PDF's file name
    filename = os.path.basename(input_file)
    
    personalId=1

    # Use Tika to parse the PDF
    parsedPDF = parser.from_file(input_file)
    # Extract the text content from the parsed PDF
    resume = parsedPDF["content"]
    # Convert double newlines into single newlines
    resume = resume.replace('\n\n', '\n')

    # Create a Pandas DataFrame from the lines of text in the Education table
    education_df = education_df(resume, education_pattern, college_pattern, college_columns, personalId)
    user_df = user_df(resume, user_columns, personalId)
    
user_df.head()

Name : Edward Smith
ned-smith@kellogg.northwestern.edu


,ID,First Name,Last Name,Email
0,1,Edward,Smith,ned-smith@kellogg.northwestern.edu


# Create and Transfer Extracted Data into Database

In [5]:
#In Python: Have chosen username for my computer (CHANGE IT BELOW). 
dbname = 'VANGUARD_DB'
username = 'mrr-phys'

In [6]:
## 'engine' is a connection to a database
## Here, I'm using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print engine.url

postgres://mrr-phys@localhost/VANGUARD_DB


In [7]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


In [8]:
## transfer the datasets into database 
user_df.to_sql('user_table', engine, if_exists='replace')
education_df.to_sql('education_table', engine, if_exists='replace')

In [9]:
# connect:
con = None
con = psycopg2.connect(database = dbname, user = username)

# query:
sql_query = """
SELECT * FROM education_table;
"""
user_data_from_sql = pd.read_sql_query(sql_query,con)

user_data_from_sql.head()

,index,ID,School,State,From_year,To_year
0,0,1,"University of Chicago, Booth School of Business","Chicago, IL",2005,2010
1,1,1,Yale University New,"Haven, CT",1999,2003
2,2,1,Capital University of Economics and Business,"Beijing, China",2001,None


*** In conclusion, I parsed the pdf with Tika module and extracted the education information and first/last name. I stored these data in PostGreSQL as two tables of user and education. Finding first and last name could be approached by using name entity recognition/analyzing the "chuncked" text data. Initially I followed this path but it was still not reliable enough to get the initial names correctlly for this case. I believe the information in the email address and metadata of the pdf file would be useful for future studies to detect the first/last name more accurately. Besides Tika, there is another PDFMiner parser that is a powerful module since it gives great access to interpreters and layout of pdf files, I will consider using it for the future analysis. ***